<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentatorResults/blob/main/rawDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook generates or makes available the links of the following data used for exploring TotalSegmentator Results on NLST


- `PerframeFunctionalGroupsSequence`: Bigquery has a limit of 1 MB per value in a cell and DICOM Segmentation Objects generated from TotalSegmentator/dcmqi contained DICOM attribute PerFrameFunctionalGroupsSequence over 1 MB, leading to this attribute dropping out of bigquery metadata table To extract this attribute we developed a workflow on [Terra](https://dockstore.org/my-workflows/github.com/ImagingDataCommons/CloudSegmentator/perFrameFunctionalGroupSequenceExtractionOnTerra). We unnest this attribute and are making it available to public as a parquet file.

- `jsonRadiomics`: While extracting the 28 radiomics features mentioned in the paper, we also saved the raw output from pyradiomics into a json file. pyradiomics provides `general features` also along with any first or shape features. We extracted and pooled the radiomics features in json files and provided them as a parquet file. To generate this file, we used a Terra workflow, pushed the combined table to bigquery, and then exported as a parquet files.

- `bodyPartAndLaterality`: Is an intermediate table which contains info about the bodyPart segmented by TotalSegmentator, segment_number, source CT series, and its Laterality

- `segmentation_completeness_check`: Checks whether or not a segment contains atleast one slice below and above the segmenation

- `presence_of_vertebra_on_every_slice`: Checks whether or not a vertebra was present on every slice in a scan

- `laterality_check`: Checks if laterality (left vs right) is correctly assigned by TotalSegmentator



Note: We ran this notebook on a 64 vCPU, 256 GB RAM [JetStream2](https://jetstream-cloud.org/) instance using local colab runtime as described [here](https://research.google.com/colaboratory/local-runtimes.html). While some steps can be run a free colab instance (2 vCPUs, 13 GB RAM), some steps that require scanning and holding the entire data in memory are RAM intensive.


Download the bases time in parquet file format containing DICOM attribute `PerFrameFunctionalGroupsSequence` attribute and `jsonRadiomics` table

In [ ]:
!wget -q https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/nlst_totalseg_perframe.parquet
!wget -q https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/json_radiomics.parquet

### Install duckdb
duckdb is an in-memomy process database that can work with highly complex data with very small footprint. Link to their GitHub repo: https://github.com/duckdb/duckdb

In [ ]:
%%capture
!pip install --upgrade duckdb

We will be querying agains the IDC tables stored in AWS buckets in parquet files directly. Even though these tables are public, we must create a dummy secret to enable querying via duckdb

In [ ]:
import duckdb
gcs_public_sql='''
CREATE SECRET (
    TYPE GCS,
    KEY_ID '',
    SECRET ''
)'''
duckdb.query(gcs_public_sql)

## Generate Segmentation Completeness check table.
Let's do this in three steps:
1. Extract extract the bodyPart and laterality from dicom_all table
2. Get minimum and maximum z component in a series
3. Get minimum and maximum z compoent for a bodyPart

Combine the three subsets to determine segmentation completeness based on presence of at least one slice below and above the bodyPart




![Alt text](https://raw.githubusercontent.com/vkt1414/CloudSegmentatorResults/main/src/images/segmentation_completeness_check.png
)

Depending on the compute available, this step may take anywhere from 5 mins on a (64 vCPUs) or ~2 hrs (8 vCPUs) on colab pro or ~8-10 hrs (estimated) on colab free version (2 vCPUs)





In [ ]:
import subprocess
import pandas as pd
import duckdb
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def process_file(file_path):
    sql = f'''
    SELECT
    StudyInstanceUID,
    rss.SeriesInstanceUID rssSIUD,
    ss.segmentDescription,
    REGEXP_EXTRACT(ss.segmentDescription,': (.*)' , 1) totalsegSegmentLabel,
    ss.SegmentNumber segment_number,
    ss_sptcs.CodeMeaning bodyPart,
    ss_sptcs_sptmcs.CodeMeaning laterality
    FROM read_parquet('{file_path}')
    LEFT JOIN UNNEST(ReferencedSeriesSequence) t1(rss) ON TRUE
    LEFT JOIN UNNEST(SegmentSequence) t2(ss) ON TRUE
    LEFT JOIN UNNEST(ss.SegmentedPropertyTypeCodeSequence) t3 (ss_sptcs) ON TRUE
    LEFT JOIN UNNEST(ss_sptcs.SegmentedPropertyTypeModifierCodeSequence) t4(ss_sptcs_sptmcs) ON TRUE
    WHERE
    collection_id  IN ('nlst')
    and MODALITY IN ('SEG')
    and analysis_result_id IN ('TotalSegmentator-CT-Segmentations')
    '''
    return duckdb.query(sql).df()

# Define the GCS bucket directory
bucket_dir = 'gs://bq_export_idc/bigquery_export/idc_v18/dicom_all/'

# Use gsutil ls to get the list of files
proc = subprocess.run(['gsutil', 'ls', bucket_dir], capture_output=True, text=True)
files = proc.stdout.splitlines()

# Use a ProcessPoolExecutor to process files in parallel
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_file, files), total=len(files)))

# Concatenate all the results into a single DataFrame
bodyPartAndLaterality = pd.concat(results)
bodyPartAndLaterality


100%|██████████| 5001/5001 [05:16<00:00, 15.81it/s]


,StudyInstanceUID,rssSIUD,segmentDescription,totalsegSegmentLabel,segment_number,bodyPart,laterality
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.283584872380...,1.3.6.1.4.1.14519.5.2.1.7009.9004.594299167694...,TotalSegmentator 15 : lung_upper_lobe_right,lung_upper_lobe_right,14,Upper lobe of lung,Right
1,1.2.840.113654.2.55.12530111708225589708623715...,1.2.840.113654.2.55.23696367762517311761791651...,TotalSegmentator 14 : lung_lower_lobe_left,lung_lower_lobe_left,14,Lower lobe of lung,Left
2,1.2.840.113654.2.55.12530111708225589708623715...,1.2.840.113654.2.55.23696367762517311761791651...,TotalSegmentator 70 : rib_right_1,rib_right_1,53,First rib,Right
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.797190834968...,1.3.6.1.4.1.14519.5.2.1.7009.9004.126198274629...,TotalSegmentator 79 : rib_right_10,rib_right_10,65,Tenth rib,Right
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.275725030103...,1.3.6.1.4.1.14519.5.2.1.7009.9004.254705695638...,TotalSegmentator 64 : rib_left_7,rib_left_7,49,Seventh rib,Left
...,...,...,...,...,...,...,...
2168,1.2.840.113654.2.55.24743520996724964481035604...,1.2.840.113654.2.55.75891416193884470011646121...,TotalSegmentator 30 : vertebrae_T5,vertebrae_T5,25,T5 vertebra,None
2169,1.2.840.113654.2.55.24743520996724964481035604...,1.2.840.113654.2.55.75891416193884470011646121...,TotalSegmentator 45 : heart_atrium_left,heart_atrium_left,34,Left atrium,None
2170,1.2.840.113654.2.55.26096434357414097924788200...,1.2.840.113654.2.55.26122393554409295038882991...,TotalSegmentator 34 : vertebrae_T1,vertebrae_T1,30,T1 vertebra,None
2171,1.2.840.113654.2.55.18798690957573916889401685...,1.2.840.113654.2.55.19300254091202538238567307...,TotalSegmentator 28 : vertebrae_T7,vertebrae_T7,23,T7 vertebra,None


This subset is useful in the next check as well and so we will save it as a parquet file, which we also made it available on GitHub

In [ ]:
bodyPartAndLaterality.to_parquet('bodyPartAndLaterality.parquet', compression='zstd')

If you would like to skip the previous step, you can download the file we made available on GitHub

In [ ]:
#!wget https://github.com/vkt1414/CloudSegmentatorResults/releases/download/0.0.1/bodyPartAndLaterality.parquet

This step retrieves the z positions of the terminal slices of a segmentation

In [ ]:
body_parts_min_max_z_values_sql='''
  SELECT
    ReferencedSeriesSequence_SeriesInstanceUID rssSIUD,
    SegmentIdentificationSequence_ReferencedSegmentNumber segment_number,
    MAX(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) segmentation_max_z_position,
    MIN(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) segmentation_min_z_position
  FROM
    'nlst_totalseg_perframe.parquet' pffgs
  GROUP BY
    rssSIUD,
    segment_number
'''
body_parts_min_max_z_values=duckdb.sql(body_parts_min_max_z_values_sql).df()
body_parts_min_max_z_values

,rssSIUD,segment_number,segmentation_max_z_position,segmentation_min_z_position
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.242007427633...,3,-68.0,-250.00
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.242007427633...,29,-113.0,-167.00
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.242020270398...,22,-253.5,-309.50
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.242094777622...,14,-78.5,-298.50
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.242094777622...,20,-222.5,-269.50
...,...,...,...,...
9565700,1.3.6.1.4.1.14519.5.2.1.7009.9004.188071475950...,62,-149.0,-262.00
9565701,1.3.6.1.4.1.14519.5.2.1.7009.9004.188071475950...,9,-261.0,-322.00
9565702,1.3.6.1.4.1.14519.5.2.1.7009.9004.188071475950...,20,-261.0,-314.00
9565703,1.3.6.1.4.1.14519.5.2.1.7009.9004.190645578368...,71,437.0,410.00


This step retrieves the z-positions of the terminal slices of a series or scan as a whole

In [ ]:
series_min_max_z_values_sql='''
  SELECT
    ReferencedSeriesSequence_SeriesInstanceUID rssSIUD,
    MAX(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) series_max_z_position,
    MIN(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) series_min_z_position
  FROM
    'nlst_totalseg_perframe.parquet' pffgs
  GROUP BY
    rssSIUD
'''
series_min_max_z_values=duckdb.sql(series_min_max_z_values_sql).df()
series_min_max_z_values

,rssSIUD,series_max_z_position,series_min_z_position
0,1.2.840.113654.2.55.28746293603353987989648661...,45.230011,-339.769989
1,1.2.840.113654.2.55.28847801925487606621174246...,4.519989,-285.480011
2,1.2.840.113654.2.55.28903247969007649577581995...,10.480072,-307.019928
3,1.2.840.113654.2.55.28918560217007925611005117...,-0.480011,-295.480011
4,1.2.840.113654.2.55.29013445046797815709310412...,13.355011,-296.644989
...,...,...,...
126048,1.3.6.1.4.1.14519.5.2.1.7009.9004.210551425060...,34.125000,-329.875000
126049,1.3.6.1.4.1.14519.5.2.1.7009.9004.208879238832...,486.000000,123.000000
126050,1.3.6.1.4.1.14519.5.2.1.7009.9004.209843747414...,370.000000,13.000000
126051,1.3.6.1.4.1.14519.5.2.1.7009.9004.211494825952...,38.250000,-299.750000


Finally, we put the previous three tables together to determine if there were any slices below and above a segmentation, to evaluate segmentation completeness check

In [ ]:
segmentation_completeness_check_sql='''
SELECT
  DISTINCT series_min_max_z_values.rssSIUD,
  segmentDescription,
  bodyPartAndLaterality.totalsegSegmentLabel,
  bodyPartAndLaterality.segment_number,
  bodyPartAndLaterality.bodyPart,
  bodyPartAndLaterality.laterality,
  series_min_max_z_values.series_max_z_position,
  series_min_max_z_values.series_min_z_position,
  body_parts_min_max_z_values.segmentation_max_z_position,
  body_parts_min_max_z_values.segmentation_min_z_position,
  CASE
    WHEN ((ABS(series_max_z_position-segmentation_max_z_position)>0 AND ABS(series_min_z_position-segmentation_min_z_position)>0)) THEN 'may_be_segmented_fully'
  ELSE
  'may_not_be_segmented_fully'
END
  AS segmentation_completeness_check,
FROM
  'bodyPartAndLaterality.parquet' bodyPartAndLaterality
JOIN
  series_min_max_z_values
ON
  bodyPartAndLaterality.rssSIUD=series_min_max_z_values.rssSIUD
JOIN
  body_parts_min_max_z_values
ON
  bodyPartAndLaterality.rssSIUD=body_parts_min_max_z_values.rssSIUD
  AND bodyPartAndLaterality.segment_number= body_parts_min_max_z_values.segment_number
'''
segmentation_completeness_check_df=duckdb.sql(segmentation_completeness_check_sql).df()
segmentation_completeness_check_df

,rssSIUD,segmentDescription,totalsegSegmentLabel,segment_number,bodyPart,laterality,series_max_z_position,series_min_z_position,segmentation_max_z_position,segmentation_min_z_position,segmentation_completeness_check
0,1.2.840.113654.2.55.23332916925935721626667482...,TotalSegmentator 103 : iliopsoas_right,iliopsoas_right,75,Iliopsoas muscle,Right,0.000000,-302.500000,-302.500000,-302.500000,may_not_be_segmented_fully
1,1.2.840.113654.2.55.26430145966415394358614907...,TotalSegmentator 85 : scapula_right,scapula_right,71,Scapula,Right,-6.625000,-321.625000,-6.625000,-169.125000,may_not_be_segmented_fully
2,1.2.840.113654.2.55.26442687222006793438873472...,TotalSegmentator 34 : vertebrae_T1,vertebrae_T1,28,T1 vertebra,None,-48.000000,-350.000000,-48.000000,-74.000000,may_not_be_segmented_fully
3,1.2.840.113654.2.55.26451528531772200283790223...,TotalSegmentator 85 : scapula_right,scapula_right,66,Scapula,Right,11.375000,-291.125000,11.375000,-153.625000,may_not_be_segmented_fully
4,1.2.840.113654.2.55.26451528531772200283790223...,TotalSegmentator 35 : vertebrae_C7,vertebrae_C7,31,C7 vertebra,None,11.375000,-291.125000,11.375000,3.875000,may_not_be_segmented_fully
...,...,...,...,...,...,...,...,...,...,...,...
9565549,1.3.6.1.4.1.14519.5.2.1.7009.9004.344492833689...,TotalSegmentator 73 : rib_right_4,rib_right_4,55,Fourth rib,Right,2.390014,-307.610016,-52.609993,-153.860001,may_be_segmented_fully
9565550,1.3.6.1.4.1.14519.5.2.1.7009.9004.285142166736...,TotalSegmentator 12 : adrenal_gland_left,adrenal_gland_left,12,Adrenal gland,Left,59.000000,-257.000000,-199.000000,-239.000000,may_be_segmented_fully
9565551,1.2.840.113654.2.55.14927309995900136148780738...,TotalSegmentator 71 : rib_right_2,rib_right_2,59,Second rib,Right,-40.000000,-375.000000,-80.000000,-125.000000,may_be_segmented_fully
9565552,1.2.840.113654.2.55.12405247702512509063214544...,TotalSegmentator 30 : vertebrae_T5,vertebrae_T5,25,T5 vertebra,None,-16.114990,-323.614990,-108.614990,-161.114990,may_be_segmented_fully


In [ ]:
segmentation_completeness_check_df.to_parquet('segmentation_completeness_table.parquet', compression='zstd')

## Generate presence of vertebrae on every slice in a series check table.

This is done in 3 steps:
1. Extract the bodyPart and laterality from dicom_all table. We will reuse the bodyPartAndLaterality dataframe generated in the previous step
2. Then, we retreive the slices where the vertebrae are present.
3. Count the number of slices present in the scan, and compare if vertebrae are present on all slices or not.

You can download the bodyPartandLaterality data from GitHub if needed

In [ ]:
#!wget https://github.com/vkt1414/CloudSegmentatorResults/releases/download/0.0.1/bodyPartAndLaterality.parquet

We will use idc-index python package to retrieve some series level info. Learn more about idc-index at https://github.com/ImagingDataCommons/idc-index

In [ ]:
%%capture
!pip install idc-index

In [ ]:
from idc_index import index
client=index.IDCClient()

Lets rename index in idc_index as idc_index_df so we can use it for queries using duckdb

In [ ]:
idc_index_df=client.index

In [ ]:
import duckdb
presence_of_vertebra_check_sql='''
WITH
  slicePositions AS (
  SELECT
    ReferencedSeriesSequence_SeriesInstanceUID rssSIUD,
    SegmentIdentificationSequence_ReferencedSegmentNumber segment_number,
    CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT) z_position,
    bodyPartAndLaterality.StudyInstanceUID,
    bodyPartAndLaterality.SegmentDescription,
    bodyPartAndLaterality.bodyPart,
    bodyPartAndLaterality.laterality,
  FROM
    'nlst_totalseg_perframe.parquet' pffgs
  JOIN
    'bodyPartAndLaterality.parquet'
  ON
    pffgs.ReferencedSeriesSequence_SeriesInstanceUID = bodyPartAndLaterality.rssSIUD
    AND pffgs.SegmentIdentificationSequence_ReferencedSegmentNumber = bodyPartAndLaterality.segment_number
  WHERE
    totalsegSegmentLabel LIKE '%vertebra%' ),
  slice_counts AS(
  SELECT
    slicePositions.rssSIUD,
    COUNT(DISTINCT z_position) slice_count_with_vertebrae
  FROM
    slicePositions
  GROUP BY
    slicePositions.rssSIUD ),
  sopInstanceCounts AS (
  SELECT
    StudyInstanceUID,
    SeriesInstanceUID,
    seriesNumber,
    instanceCount
  FROM
    idc_index_df
  WHERE
    collection_id IN ('nlst')
    AND MODALITY IN ('CT') )
SELECT
  DISTINCT SeriesInstanceUID,
  instanceCount,
  slice_count_with_vertebrae,
  (instanceCount-slice_counts.slice_count_with_vertebrae) difference,
  CONCAT('https://viewer.imaging.datacommons.cancer.gov/viewer/',StudyInstanceUID) viewerUrl,
  SeriesNumber
FROM
  slice_counts
LEFT JOIN
  sopInstanceCounts
ON
  slice_counts.rssSIUD=sopInstanceCounts.SeriesInstanceUID
ORDER BY
  difference desc
'''
presence_of_vertebra_on_every_slice_check=duckdb.sql(presence_of_vertebra_check_sql).df()
presence_of_vertebra_on_every_slice_check


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,SeriesInstanceUID,instanceCount,slice_count_with_vertebrae,difference,viewerUrl,SeriesNumber
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.145914250167...,716,558,158,https://viewer.imaging.datacommons.cancer.gov/...,4
1,1.2.840.113654.2.55.11098174635431959321326652...,156,1,155,https://viewer.imaging.datacommons.cancer.gov/...,3
2,1.2.840.113654.2.55.16186212658523972455256118...,361,227,134,https://viewer.imaging.datacommons.cancer.gov/...,7
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.235844760638...,730,597,133,https://viewer.imaging.datacommons.cancer.gov/...,4
4,1.2.840.113654.2.55.31102992820127351566476421...,147,20,127,https://viewer.imaging.datacommons.cancer.gov/...,3
...,...,...,...,...,...,...
126032,1.2.840.113654.2.55.45955178010319073836110821...,182,182,0,https://viewer.imaging.datacommons.cancer.gov/...,4
126033,1.2.840.113654.2.55.28260276387596284881934210...,130,130,0,https://viewer.imaging.datacommons.cancer.gov/...,2
126034,1.3.6.1.4.1.14519.5.2.1.7009.9004.314090928267...,172,172,0,https://viewer.imaging.datacommons.cancer.gov/...,2
126035,1.3.6.1.4.1.14519.5.2.1.7009.9004.272175156809...,189,189,0,https://viewer.imaging.datacommons.cancer.gov/...,3


Lets also write this to a parquet file, so we can use it again in later steps

In [ ]:
presence_of_vertebra_on_every_slice_check.to_parquet('vertabra-at-every-slice-check-table.parquet', compression='zstd')


## Check if Laterality was correctly assigned by TotalSegmentator
We checked by comparing the x coordinate of pyradiomics general module feature Center of Mass. Because we did not save general module features into a DICOM Structured Reports, we made them available as a parquet file instead.

In [ ]:
laterality_check_sql='''
with temp as
(SELECT
  CT_seriesInstanceUID,
  organ,
  REGEXP_REPLACE(REGEXP_REPLACE(organ, 'left', ''), 'right','') as organ_with_out_laterality,
  diagnostics_Mask_original_CenterOfMass[1] AS x_coordinate,
  CASE
  WHEN organ like '%left%' THEN 'left'
  wHEN organ like '%right%' THEN 'right'
  else null
  end as laterality
FROM
  'json_radiomics.parquet'
),
temp2 as (
select
CT_seriesInstanceUID,
organ organ_left,
x_coordinate x_coordinate_left,
lead(organ) over (partition by CT_SeriesInstanceUID, organ_with_out_laterality order by organ) organ_right,
lead(x_coordinate) over (partition by CT_SeriesInstanceUID, organ_with_out_laterality order by organ) x_coordinate_right
from temp where laterality is not null
order by 1,2
)
select distinct
temp2.*,
CASE WHEN x_coordinate_left > x_coordinate_right THEN 'pass'
     ELSE 'fail'
END AS laterality_check,
from temp2
where organ_right is not null

'''
laterality_check=duckdb.sql(laterality_check_sql).df()
laterality_check

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CT_SeriesInstanceUID,organ_left,x_coordinate_left,organ_right,x_coordinate_right,laterality_check
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.867694349870...,scapula_left,114.680100,scapula_right,-135.690404,pass
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.867908906957...,adrenal_gland_left,35.197955,adrenal_gland_right,-44.661399,pass
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.867908906957...,rib_left_1,69.302722,rib_right_1,-51.022865,pass
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.867943465381...,clavicula_left,43.940431,clavicula_right,-16.741126,pass
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.867943465381...,rib_left_6,116.397618,rib_right_6,-85.460047,pass
...,...,...,...,...,...,...
2822843,1.3.6.1.4.1.14519.5.2.1.7009.9004.304205637634...,rib_left_11,90.579126,rib_right_11,-71.418031,pass
2822844,1.3.6.1.4.1.14519.5.2.1.7009.9004.304207227350...,clavicula_left,42.415221,clavicula_right,-37.390977,pass
2822845,1.3.6.1.4.1.14519.5.2.1.7009.9004.304207227350...,heart_ventricle_left,49.953274,heart_ventricle_right,21.364797,pass
2822846,1.3.6.1.4.1.14519.5.2.1.7009.9004.304207227350...,rib_left_1,57.037112,rib_right_1,-56.087659,pass


Lets write the results to a parquet file

In [ ]:
laterality_check.to_parquet('laterality_check_table.parquet', compression='zstd')


## Qualitative Checks Table

In this table, we combined the results of all three checks above, and pulled pyradiomics general features voxel num and connected volumes attributes.

In [ ]:
qual_checks_table='''
WITH temp as(
SELECT
  DISTINCT
  rssSIUD AS CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  idc.seriesNumber,
  bodyPart,
  sct.laterality,
  laterality_check,
  CASE
    WHEN sct.segmentation_completeness_check ='may_be_segmented_fully' THEN 'pass'
    ELSE 'fail'
  END AS segmentation_completeness,
  CASE
    WHEN vaesct.difference =0 THEN 'pass'
    WHEN vaesct.difference !=0 THEN 'fail'
    ELSE null
  END AS series_with_vertabra_on_every_slice,
jr.diagnostics_Mask_original_VolumeNum as connected_volumes,
  diagnostics_Mask_original_VoxelNum as voxel_num
FROM
  '/content/segmentation_completeness_table.parquet' sct
LEFT JOIN
  'json_radiomics.parquet' jr
ON
  sct.rssSIUD=jr.CT_seriesInstanceUID
AND sct.totalsegSegmentLabel= jr.organ

LEFT JOIN
'vertabra-at-every-slice-check-table.parquet' vaesct
ON vaesct.SeriesInstanceUID=sct.rssSIUD

JOIN idc_index_df idc
ON idc.SeriesInstanceUID = sct.rssSIUD

LEFT JOIN 'laterality_check_table.parquet' lct
  ON sct.rssSIUD = lct.CT_SeriesInstanceUID
  and sct.totalsegSegmentLabel = lct.organ_left
),
temp2 as(
SELECT *  FROM temp
UNION BY NAME
SELECT distinct
  temp.CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  seriesNumber,
  bodyPart,
  laterality,
  segmentation_completeness,
  series_with_vertabra_on_every_slice,
  connected_volumes,
  voxel_num,
  lct.laterality_check
FROM
temp
LEFT JOIN 'laterality_check_table.parquet' lct
  ON temp.CT_SeriesInstanceUID = lct.CT_SeriesInstanceUID
  and temp.totalsegSegmentLabel = lct.organ_right
)
SELECT
  CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  seriesNumber,
  bodyPart,
  laterality,
  segmentation_completeness,
  series_with_vertabra_on_every_slice,
  connected_volumes,
  voxel_num,
  max(laterality_check) laterality_check
FROM
  temp2
group by 1,2,3,4,5,6,7,8,9,10
'''
qual_checks_table=duckdb.sql(qual_checks_table).df()
qual_checks_table


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CT_SeriesInstanceUID,totalsegSegmentLabel,StudyInstanceUID,SeriesNumber,bodyPart,laterality,segmentation_completeness,series_with_vertabra_on_every_slice,connected_volumes,voxel_num,laterality_check
0,1.2.840.113654.2.55.47261886645209027724254463...,iliopsoas_left,1.2.840.113654.2.55.21475346664571344367770741...,2,Iliopsoas muscle,Left,fail,pass,1.0,3112.0,pass
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.145090463630...,rib_left_1,1.3.6.1.4.1.14519.5.2.1.7009.9004.173578443251...,4,First rib,Left,pass,pass,1.0,14037.0,pass
2,1.2.840.113654.2.55.27990232972457772862383689...,colon,1.2.840.113654.2.55.29219101530520383442564352...,6,Colon,None,fail,pass,3.0,169728.0,None
3,1.2.840.113654.2.55.96136604464370611511493630...,adrenal_gland_right,1.2.840.113654.2.55.25215987855378640689496235...,2,Adrenal gland,Right,pass,pass,1.0,4152.0,pass
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.190834165393...,rib_left_9,1.3.6.1.4.1.14519.5.2.1.7009.9004.624669921179...,7067,Ninth rib,Left,pass,pass,1.0,19649.0,pass
...,...,...,...,...,...,...,...,...,...,...,...
9565549,1.2.840.113654.2.55.32405632639159587560580937...,gallbladder,1.2.840.113654.2.55.12545181757265862524798116...,4,Gallbladder,None,fail,pass,1.0,8274.0,None
9565550,1.2.840.113654.2.55.17608676141677484163790269...,vertebrae_L1,1.2.840.113654.2.55.20768113407397155831546266...,2,L1 vertebra,None,fail,pass,1.0,68039.0,None
9565551,1.2.840.113654.2.55.24513683921837088385860584...,inferior_vena_cava,1.2.840.113654.2.55.33889351046033786374166752...,1,Inferior vena cava,None,fail,pass,1.0,39164.0,None
9565552,1.2.840.113654.2.55.31488665378781239799020392...,vertebrae_T7,1.2.840.113654.2.55.12059595696735662636420926...,5,T7 vertebra,None,pass,pass,1.0,38582.0,None


Lets write the output to a parquet file

In [ ]:
qual_checks_table.to_parquet('qual_checks_table.parquet', compression='zstd')

## Flatenned quantitative measurements table

Because we are pivoting the table we use aggregate functions, and aggregate queries must go through all rows before returning any results. This resulted in extremely high RAM usage.

In [ ]:
sql='''
SELECT
  PatientID,
  segmentationInstanceUID,
  sourceSegmentedSeriesUID,
  findingSite.CodeMeaning bodyPart,
  lateralityModifier.CodeMeaning Laterality,
  MAX(CASE WHEN Quantity.CodeMeaning = '10th percentile' THEN Value END) AS "10th percentile",
  MAX(CASE WHEN Quantity.CodeMeaning = '90th percentile' THEN Value END) AS "90th percentile",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Elongation' THEN Value END) AS "Elongation",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Energy' THEN Value END) AS "Energy",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Flatness' THEN Value END) AS "Flatness",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Intensity Histogram Entropy' THEN Value END) AS "Intensity Histogram Entropy",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Intensity histogram uniformity' THEN Value END) AS "Intensity histogram uniformity",
FROM
  'gs://bq_export_idc/bigquery_export/idc_v18/quantitative_measurements/*.parquet'
WHERE
  SeriesDescription LIKE '%TotalSeg%'
GROUP BY
  PatientID, segmentationInstanceUID, sourceSegmentedSeriesUID, findingSite.CodeMeaning, lateralityModifier.CodeMeaning
'''
flattened_quantitative_measurements_1=duckdb.sql(sql).to_parquet('flattened_quantitative_measurements_1.parquet', compression='zstd')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
sql='''
SELECT
  PatientID,
  segmentationInstanceUID,
  sourceSegmentedSeriesUID,
  findingSite.CodeMeaning bodyPart,
  lateralityModifier.CodeMeaning Laterality,
  MAX(CASE WHEN Quantity.CodeMeaning = 'Interquartile range' THEN Value END) AS "Interquartile range",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Kurtosis' THEN Value END) AS "Kurtosis",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Least Axis in 3D Length' THEN Value END) AS "Least Axis in 3D Length",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Major Axis in 3D Length' THEN Value END) AS "Major Axis in 3D Length",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Maximum 3D Diameter of a Mesh' THEN Value END) AS "Maximum 3D Diameter of a Mesh",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Maximum grey level' THEN Value END) AS "Maximum grey level",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Mean' THEN Value END) AS "Mean"
FROM
  'gs://bq_export_idc/bigquery_export/idc_v18/quantitative_measurements/*.parquet'
WHERE
  SeriesDescription LIKE '%TotalSeg%'
GROUP BY
  PatientID, segmentationInstanceUID, sourceSegmentedSeriesUID, findingSite.CodeMeaning, lateralityModifier.CodeMeaning
'''
flattened_quantitative_measurements_2=duckdb.sql(sql).to_parquet('flattened_quantitative_measurements_2.parquet', compression='zstd')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
sql='''
SELECT
  PatientID,
  segmentationInstanceUID,
  sourceSegmentedSeriesUID,
  findingSite.CodeMeaning bodyPart,
  lateralityModifier.CodeMeaning Laterality,
  MAX(CASE WHEN Quantity.CodeMeaning = 'Mean absolute deviation' THEN Value END) AS "Mean absolute deviation",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Median' THEN Value END) AS "Median",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Minimum grey level' THEN Value END) AS "Minimum grey level",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Minor Axis in 3D Length' THEN Value END) AS "Minor Axis in 3D Length",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Range' THEN Value END) AS "Range",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Robust mean absolute deviation' THEN Value END) AS "Robust mean absolute deviation",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Root mean square' THEN Value END) AS "Root mean square",
FROM
  'gs://bq_export_idc/bigquery_export/idc_v18/quantitative_measurements/*.parquet'
WHERE
  SeriesDescription LIKE '%TotalSeg%'
GROUP BY
  PatientID, segmentationInstanceUID, sourceSegmentedSeriesUID, findingSite.CodeMeaning, lateralityModifier.CodeMeaning
'''
flattened_quantitative_measurements_3=duckdb.sql(sql).to_parquet('flattened_quantitative_measurements_3.parquet', compression='zstd')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
sql='''
SELECT
  PatientID,
  segmentationInstanceUID,
  sourceSegmentedSeriesUID,
  findingSite.CodeMeaning bodyPart,
  lateralityModifier.CodeMeaning Laterality,
  MAX(CASE WHEN Quantity.CodeMeaning = 'Skewness' THEN Value END) AS "Skewness",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Sphericity' THEN Value END) AS "Sphericity",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Surface Area of Mesh' THEN Value END) AS "Surface Area of Mesh",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Surface to Volume Ratio' THEN Value END) AS "Surface to Volume Ratio",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Variance' THEN Value END) AS "Variance",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Volume from Voxel Summation' THEN Value END) AS "Volume from Voxel Summation",
  MAX(CASE WHEN Quantity.CodeMeaning = 'Volume of Mesh' THEN Value END) AS "Volume of Mesh"
FROM
  'gs://bq_export_idc/bigquery_export/idc_v18/quantitative_measurements/*.parquet'
WHERE
  SeriesDescription LIKE '%TotalSeg%'
GROUP BY
  PatientID, segmentationInstanceUID, sourceSegmentedSeriesUID, findingSite.CodeMeaning, lateralityModifier.CodeMeaning
'''
flattened_quantitative_measurements_4=duckdb.sql(sql).to_parquet('flattened_quantitative_measurements_4.parquet', compression='zstd')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Combine all four parts into one

In [ ]:
import duckdb
sql='''
WITH temp AS (
SELECT * FROM 'flattened_quantitative_measurements_1.parquet'
UNION BY NAME
SELECT * FROM 'flattened_quantitative_measurements_2.parquet'
UNION BY NAME
SELECT * FROM 'flattened_quantitative_measurements_3.parquet'
UNION BY NAME
SELECT * FROM 'flattened_quantitative_measurements_4.parquet'
)
SELECT
  PatientID,
  segmentationInstanceUID,
  sourceSegmentedSeriesUID,
  bodyPart,
  Laterality,
  MAX("10th percentile") "10th percentile",
  MAX("90th percentile") "90th percentile",
  MAX("Elongation") "Elongation",
  MAX("Energy") "Energy",
  MAX("Flatness") "Flatness",
  MAX("Intensity Histogram Entropy") "Intensity Histogram Entropy",
  MAX("Intensity histogram uniformity") "Intensity histogram uniformity",
  MAX("Interquartile range") "Interquartile range",
  MAX("Kurtosis") "Kurtosis",
  MAX("Least Axis in 3D Length") "Least Axis in 3D Length",
  MAX("Major Axis in 3D Length") "Major Axis in 3D Length",
  MAX("Maximum 3D Diameter of a Mesh") "Maximum 3D Diameter of a Mesh",
  MAX("Maximum grey level") "Maximum grey level",
  MAX("Mean") "Mean",
  MAX("Mean absolute deviation") "Mean absolute deviation",
  MAX("Median") "Median",
  MAX("Minimum grey level") "Minimum grey level",
  MAX("Minor Axis in 3D Length") "Minor Axis in 3D Length",
  MAX("Range") "Range",
  MAX("Robust mean absolute deviation") "Robust mean absolute deviation",
  MAX("Root mean square") "Root mean square",
  MAX("Skewness") "Skewness",
  MAX("Sphericity") "Sphericity",
  MAX("Surface Area of Mesh") "Surface Area of Mesh",
  MAX("Surface to Volume Ratio") "Surface to Volume Ratio",
  MAX("Variance") "Variance",
  MAX("Volume from Voxel Summation") "Volume from Voxel Summation",
  MAX("Volume of Mesh") "Volume of Mesh"
FROM temp
GROUP BY 1,2,3,4,5
'''
flattened_quantitative_measurements=duckdb.sql(sql).to_parquet('flat_quantitative_measurements.parquet', compression='zstd')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Qualitative checks and quantitative measurements
Finally, lets put qualitative checks and quantitative measurements together.

In [ ]:
import duckdb

In [ ]:
sql='''
SELECT
PatientID,
--qct.*exclude (CT_SeriesInstanceUID),
qct.*,
qmt.*exclude(PatientID,segmentationInstanceUID,sourceSegmentedSeriesUID, bodyPart, Laterality)
from

'qual_checks_table.parquet' qct

join

'flat_quantitative_measurements.parquet' qmt

on qct.CT_seriesInstanceUID=qmt.sourceSegmentedSeriesUID
and qct.bodyPart=qmt.bodyPart
and ((qct.laterality = qmt.laterality) OR (qct.laterality IS NULL AND qmt.laterality IS NULL))
'''
duckdb.sql(sql).to_parquet('qual_checks_and_quantitative_measurements.parquet', compression='zstd')

#duckdb.sql(sql).df().columns

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))